<a href="https://colab.research.google.com/github/makler322/msu_ml_spring_2021/blob/master/Project/Project_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Подключим всё необходимое

In [170]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

## Посмотрим на данные

In [171]:
docs_titles_path = '/content/drive/MyDrive/Boost/docs_titles.tsv'
train_data = pd.read_csv('/content/drive/MyDrive/Boost/train_groups.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Boost/test_groups.csv')
docs_titles_data = pd.read_csv(docs_titles_path, sep='\t')

In [172]:
print(train_data.head(3))
print(train_data.info())
print(test_data.head(3))
print(test_data.info())
print(docs_titles_data.head(3))
print(docs_titles_data.info())

   pair_id  group_id  doc_id  target
0        1         1   15731       0
1        2         1   14829       0
2        3         1   15764       0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11690 entries, 0 to 11689
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   pair_id   11690 non-null  int64
 1   group_id  11690 non-null  int64
 2   doc_id    11690 non-null  int64
 3   target    11690 non-null  int64
dtypes: int64(4)
memory usage: 365.4 KB
None
   pair_id  group_id  doc_id
0    11691       130    6710
1    11692       130    4030
2    11693       130    5561
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16627 entries, 0 to 16626
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   pair_id   16627 non-null  int64
 1   group_id  16627 non-null  int64
 2   doc_id    16627 non-null  int64
dtypes: int64(3)
memory usage: 389.8 KB
None
   doc_id              

### В этих данных только заголовки, а ещё у нас есть архив на 6гб с html страницами, где лежит очень много данных. Нужно будет его распарсить. Очевидно, что с ними у нас любой алогритм заработает лучше. Поэтому задача парсинга - одна из главных.

## Предобработка данных

### Создадим словарик: id - заголовок. По сути, просто переведём docs_titles в словарь
### После парсинга поле "заголовок" поменяется.

In [173]:
doc_to_title = {}
with open(docs_titles_path) as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print(len(doc_to_title))

28026


### Здесь сразу разобьём нашу train выборку на train/test для оценки своего промежуточного результата.

In [174]:
X_train, X_test, y_train, y_test = train_test_split(train_data.drop(['target'], axis=1), train_data[['target']], test_size=0.20, random_state=42)
print(X_train.head(3))
print(y_train.head(3))
print(X_test.head(3))
print(y_test.head(3))

      pair_id  group_id  doc_id
9927     9928       110    3001
714       715         8   17910
1076     1077        14    8507
      target
9927       0
714        0
1076       0
      pair_id  group_id  doc_id
8726     8727        98   15313
4884     4885        55   13381
5023     5024        57    5541
      target
8726       0
4884       0
5023       0


### Создадим ещё один словарь. На этот раз с группами, полученными на train.
### В качестве аргументов: (id | заголовок | target). После парсинга поменятся только заголовок

In [175]:
traingroups_titledata = {}
for i in tqdm(range(len(X_train))):
    new_doc = X_train.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = y_train.iloc[i]
    title = doc_to_title[doc_id]  # гарантировано есть
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))

### Сделаем тоже самое, но для test.

In [176]:
testgroups_titledata = {}
for i in tqdm(range(len(X_test))):
    new_doc = X_test.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    title = doc_to_title[doc_id]
    target = y_test.iloc[i]
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, title, target))  # Здесь target не используется. Просто костыль.

## Смысл идеи: для каждого сайта мы смотрим пересечение с другими по колличеству слов. Больше слов совпало - релевантнее сайт.

In [177]:

y_train = []
X_train = []
groups_train = []
for new_group in tqdm(traingroups_titledata):
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_train.append(sorted(all_dist, reverse=True)[0:15])
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print(X_train.shape, y_train.shape, groups_train.shape)
print(X_train[1083:1087])
print(y_train[1083:1087])


(9352, 15) (9352, 1) (9352,)
[[ 4  3  3  2  2  2  2  2  2  1  1  1  1  1  1]
 [10  2  2  2  2  1  1  1  1  1  1  1  1  1  1]
 [ 1  1  1  1  1  1  0  0  0  0  0  0  0  0  0]
 [ 1  1  1  0  0  0  0  0  0  0  0  0  0  0  0]]
[[1]
 [0]
 [1]
 [0]]


### Получаем данные, где для каждого вектора есть своя метка класса. С этими векторами и будем работать.
### Аналогично строим вектора для test

In [178]:
X_test1 = []
groups_test = []
for new_group in tqdm(testgroups_titledata):
    docs = testgroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        groups_test.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, max(16, len(docs))):
            if k == j:
                continue
            try:
                doc_id_j, title_j, target_j = docs[j]
                words_j = set(title_j.strip().split())
                all_dist.append(len(words.intersection(words_j)))
            except:
                all_dist.append(0)
            

        X_test1.append(sorted(all_dist, reverse=True)[0:15])

#s = pd.Series(X_test1)
#df = pd.DataFrame(s, columns=['foo'])
#X_test = pd.DataFrame(df['foo'].tolist()).values

X_test = np.array(X_test1)
groups_test = np.array(groups_test)
print(X_test.shape, groups_test.shape)
print(X_test)


(2338, 15) (2338,)
[[2 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [2 1 1 ... 0 0 0]
 ...
 [2 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [2 1 1 ... 0 0 0]]


## Возьмём рандомный классификатор и оценим его на test

In [179]:
clf = xgb.XGBClassifier()
# clf = lgb.LGBMClassifier()

In [180]:
clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

### Напомню, что у нас f1-score

In [181]:
y_pred = clf.predict(X_test)
y_pred = y_pred.astype(int)

f1_score(y_test, y_pred)

0.22263450834879406

## Отправляем

In [182]:
testgroups_title = {}
for i in tqdm(range(len(test_data))):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    title = doc_to_title[doc_id]
    if doc_group not in testgroups_title:
        testgroups_title[doc_group] = []
    testgroups_title[doc_group].append((doc_id, title))  

In [183]:
Test = []
groups_test = []
for new_group in tqdm(testgroups_title):
    docs = testgroups_title[new_group]
    for k, (doc_id, title) in enumerate(docs):
        groups_test.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, max(16, len(docs))):
            if k == j:
                continue
            try:
                doc_id_j, title_j = docs[j]
                words_j = set(title_j.strip().split())
                all_dist.append(len(words.intersection(words_j)))
            except:
                all_dist.append(0)
            

        Test.append(sorted(all_dist, reverse=True)[0:15])
Test = np.array(Test)


In [184]:
y_pred = clf.predict(Test)
y_pred = y_pred.astype(int)

In [185]:
sub = pd.DataFrame({'pair_id': np.asarray(test_data['pair_id']), 'target': y_pred})
sub = sub.set_index(['pair_id'])
sub.to_csv('sub.csv')

## Score = 0.59923